<a href="https://colab.research.google.com/github/sameer203/Tensorflow_/blob/main/11_NLP_fundamentals_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2021-11-03 03:27:38--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2021-11-03 03:27:38 (59.8 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-11-03 03:27:38--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-11-03 03:27:39 (71.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys,unzip_data

In [4]:
unzip_data("nlp_getting_started.zip")

### Visualizing Test data
1. pandas
2. tensorflowload text

In [5]:
import pandas as pd

In [6]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
# Shuffle training data
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
train_df_shuffled.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
len(train_df), len(test_df)

(7613, 3263)

In [11]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real diaster)" if target > 0 else "(not real diaster)")
  print(f"Text:\n {text}\n")

Target: 0 (not real diaster)
Text:
 @johndcgow heard this few days ago while driving and near crashed the car from laughing to much

Target: 1 (real diaster)
Text:
 Wreckage 'conclusively confirmed' as from missing flight MH370 via @YahooNewsDigest

Target: 1 (real diaster)
Text:
 Whirlwind that has lasted for more than an hour and still strong. A standstill in parts of middle east.

Target: 1 (real diaster)
Text:
 I can't believe Blevins got injured falling off a curb. That's something that would happen to me never it expected for a pro player

Target: 1 (real diaster)
Text:
 Tomorrow's evening commute receives a RED LIGHT. A rainstorm impacting the region will push it's way in late evening. http://t.co/DRvm8ISOtE



In [12]:
from sklearn.model_selection import train_test_split

In [13]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(train_df_shuffled["text"].to_numpy(), 
                                                                              train_df_shuffled["target"].to_numpy(), 
                                                                              test_size=0.1, 
                                                                              random_state=42)

In [14]:
len(train_sentences), len(test_sentences)

(6851, 762)

In [15]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

### Text Vectorization

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [17]:
# use the defaultTextVectorization parameters
text_vectorization = TextVectorization(max_tokens=None,
                                       standardize="lower_and_strip_punctuation",
                                       split="whitespace",
                                       ngrams=None,
                                       output_mode="int",
                                       output_sequence_length=None,
                                       pad_to_max_tokens=False)


In [18]:
# Find the average number of tokens in the training tweets
sum = 0
for i in train_sentences:
  sum = sum + len(i.split())
round(sum)
average = round(sum/len(train_sentences))
average

15

In [19]:
# Setup text vectorization variables
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                       output_sequence_length=max_length,
                                       output_mode="int")

In [20]:
# Fit the text vectorizer to the training data
text_vectorizer.adapt(train_sentences)

In [21]:
# Create a samplesentence and tokenize it
sample_sentence= "My name is sameer"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 13, 735,   9,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
# Vectorize a random sentence from training set
random_sentence = random.choice(train_sentences)

random_sentence, text_vectorizer([random_sentence])

('Experts in France begin examining airplane debris found on Reunion Island: French air accident experts on Wednesday began examining t...',
 <tf.Tensor: shape=(1, 15), dtype=int64, numpy=
 array([[ 854,    4, 1021, 1223, 1413,  444,  247,  225,   11,  432,  419,
         1409,  307,  128,  854]])>)

In [23]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab[:5]

['', '[UNK]', 'the', 'a', 'in']

In [24]:
words_in_vocab[-5:]

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

In [25]:
from tensorflow.keras import layers

In [26]:
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)
embedding

In [27]:
# turn sentences into dense vector of fixed size
random_sentence = random.choice(train_sentences)

sample_embed = embedding(text_vectorizer([random_sentence]))

random_sentence, sample_embed

('I could demolish this right now! https://t.co/SkS5jCCrj2',
 <tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
 array([[[-0.01607101, -0.02135834,  0.00732363, ..., -0.00471625,
           0.03118281,  0.00302887],
         [ 0.02634783, -0.04586439,  0.04802313, ..., -0.03247565,
          -0.01522174,  0.03048858],
         [-0.04617889,  0.02536081, -0.04003403, ..., -0.00767316,
          -0.04808238, -0.0289184 ],
         ...,
         [ 0.02839769, -0.03222448,  0.02900069, ..., -0.01137893,
           0.04902747,  0.0211115 ],
         [ 0.02839769, -0.03222448,  0.02900069, ..., -0.01137893,
           0.04902747,  0.0211115 ],
         [ 0.02839769, -0.03222448,  0.02900069, ..., -0.01137893,
           0.04902747,  0.0211115 ]]], dtype=float32)>)

In [28]:
# checkout single tokens embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.01607101, -0.02135834,  0.00732363,  0.04696866, -0.00504091,
        -0.02328577,  0.02964686,  0.00783492, -0.00416565,  0.03176917,
        -0.03599985,  0.01522313,  0.02403394,  0.00737737,  0.0238133 ,
        -0.03323229,  0.04780803,  0.03301871,  0.03771334,  0.00306337,
         0.02427595, -0.00299836, -0.047883  ,  0.00867657, -0.0064675 ,
         0.0350968 , -0.03761684,  0.02978611,  0.0229332 , -0.00132258,
        -0.03694048,  0.00398051, -0.02453305, -0.03570146, -0.01563806,
         0.00953735, -0.04135649,  0.04682368,  0.00395511,  0.01775228,
         0.02616346, -0.00900863,  0.01922918,  0.04193484, -0.00312452,
         0.04990845, -0.02641439,  0.01638489, -0.01191863,  0.03809736,
        -0.049296  ,  0.03414598,  0.02384497, -0.02809541, -0.0032899 ,
         0.03445499,  0.03457984, -0.03726475, -0.0105185 ,  0.0273533 ,
        -0.00851569, -0.03025404, -0.01179012, -0.03721677,  0.02072693,
  

### Model 0: Getting a baseline

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


In [32]:
# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [35]:
# Evaluate the baseline model
baseline_score = model_0.score(test_sentences, test_labels)
baseline_score*100

79.26509186351706